In [6]:
# 构建前端数据集，通过将huggingface的Tensoic/FrontendCookbook数据集翻译为中文进行
import re
import os
import json

from datasets import load_dataset
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langsmith import traceable
from langsmith.wrappers import wrap_openai

reg = r'```\w+\n[^`]+```'

llm = ChatOpenAI(openai_proxy='http://172.17.208.1:7890')
prompt = ChatPromptTemplate.from_messages([
    ("system", "你将扮演一个翻译，将我给你的文字从英文翻译为中文.你只负责翻译，不要回答问题。你也只返回翻译后结果,不要加任何其他文字"),
    ("user", "{input}")
])
chain = prompt | llm 

def question_has_code(example):
    return re.search(reg,example['question'])

@traceable
def trans_by_llama(example):
    question =  example['question']
    example['instruction'] = chain.invoke({"input": question}).content
    example['output'] = example['answer']
    example['input'] = ''
    del example['question']
    del example['answer']
    return example

dataset = load_dataset("Tensoic/FrontendCookbook", split="train")

dataset = dataset.filter(question_has_code)
dataset = dataset.map(trans_by_llama)

dataset.push_to_hub('dailin/FrontendCookbook')

Filter:   0%|          | 0/45448 [00:00<?, ? examples/s]

Parameter 'function'=<function trans_by_llama at 0x7f32e0c43130> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/3175 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dailin/FrontendCookbook/commit/d85df919d450f3fd0f203800088ff433ef506501', commit_message='Upload dataset', commit_description='', oid='d85df919d450f3fd0f203800088ff433ef506501', pr_url=None, pr_revision=None, pr_num=None)